## Importação das bibliotecas utilizadas no tratamento inicial

In [15]:
#Import das bibliotecas
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from string import punctuation
from nltk import word_tokenize
from nltk.probability import FreqDist
import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Realização da leitura do dataset Fakepedia, inspeção inicial e criação da coluna para tratamento

In [16]:
#Realizando leitura do dataset fakepedia
url = "https://raw.githubusercontent.com/andersoncordeiro/Fakepedia-Corpus/main/dataset/fakepedia-corpus-v1.csv"
df = pd.read_csv(url, header=0, sep=";")

In [17]:
#Observando estrutura inicial do dataset
df.head()

,title,title_norm,message,message_norm,tokens,features,entities,type,source,url_review
0,"Pastor Sérgio Von Helder, que chutou santa, nã...","pastor sergio von helder, que chutou santa, na...",NaN,NaN,"['pastor', 'sergio', 'von', 'helder', ',', 'qu...","{'len': 12, 1: {'token': 'pastor', 'lemma': 'p...",[Pastor Sérgio Von Helder],fake,boatos.org,https://www.boatos.org/religiao/pastor-sergio-...
1,"Lulinha, filho de Lula, comprou avião Gulfstre...","lulinha, filho de lula, comprou aviao gulfstre...",NaN,NaN,"['lulinha', ',', 'filho', 'de', 'lula', ',', '...","{'len': 14, 1: {'token': 'lulinha', 'lemma': '...","[Lula, Gulfstream]",fake,boatos.org,https://www.boatos.org/politica/lulinha-aviao-...
2,Lulinha não comprou Fazenda Fortaleza por 47 m...,lulinha nao comprou fazenda fortaleza por 47 m...,O Novo Mega Campeão do Brasil de enriqueciment...,o novo mega campeao do brasil de enriqueciment...,"['lulinha', 'nao', 'comprou', 'fazenda', 'fort...","{'len': 8, 1: {'token': 'lulinha', 'lemma': 'l...","[Lulinha, Fazenda]",fake,boatos.org,https://www.boatos.org/politica/lulinha-fazend...
3,Fanta Uva causa câncer e problema nos rins,fanta uva causa cancer e problema nos rins,ALERTA GERAL ATENÇÃO” FANTA UVA!!!! A propagan...,alerta geral atencao fanta uva!!!!a propaganda...,"['fanta', 'uva', 'causa', 'cancer', 'e', 'prob...","{'len': 8, 1: {'token': 'fanta', 'lemma': 'fan...",[Fanta Uva],fake,boatos.org,https://www.boatos.org/saude/fanta-uva-da-canc...
4,Simpsons terá episódio sobre manifestações no ...,simpsons tera episodio sobre manifestacoes no ...,NaN,NaN,"['simpsons', 'tera', 'episodio', 'sobre', 'man...","{'len': 7, 1: {'token': 'simpsons', 'lemma': '...","[Simpsons, Brasil]",fake,boatos.org,https://www.boatos.org/entretenimento/simpsons...


In [18]:
#Verificando dimensões do dataset
df.shape

(8517, 10)

In [19]:
# Criando coluna com o texto das notícias normalizadas que será usada para o tratamento
df['message_norm_treatment'] = df['message_norm']

In [20]:
#Removendo notícias duplicadas após toda limpeza usando como parâmetro o texto da notícia
testeremoveduplicata = df.drop_duplicates(subset=['message'], keep='first')
testeremoveduplicata.shape

(4205, 11)

In [21]:
#Removendo notícias duplicadas após toda limpeza usando como parâmetro o texto e o título da notícia
df = df.drop_duplicates(subset=['title','message'], keep='first')

In [22]:
#Verificando o tamanho do dataset
df.shape

(5201, 11)

In [23]:
#Pegando o dataset que contém notícias com mesmo texto mas títulos diferentes
df_sametext = pd.concat([df, testeremoveduplicata]).drop_duplicates(keep=False)
df_sametext = df_sametext.reset_index()

#### Remover as duplicatas das notícias considerando somente o texto da notícia reduziu o dataset de 8517 amostras para 4205, por outro lado remover as duplicatas das notícias considerando o texto e o título da notícia reduziu o dataset de 8517 amostras para 5201, por inferência podemos afirmar que existem notícias no dataset com mesmo texto, porém notícias diferentes.

In [24]:
df.describe()

,title,title_norm,message,message_norm,tokens,features,entities,type,source,url_review,message_norm_treatment
count,5201,5201,4684,4596,5201,5201,5201,5201,5173,5201,4596
unique,5201,5201,4204,4196,5201,5201,4016,1,1,5201,4196
top,"Pastor Sérgio Von Helder, que chutou santa, nã...","pastor sergio von helder, que chutou santa, na...",Ps.: Esse artigo é uma sugestão de leitores do...,ps.: esse artigo e uma sugestao de leitores do...,"['pastor', 'sergio', 'von', 'helder', ',', 'qu...","{'len': 12, 1: {'token': 'pastor', 'lemma': 'p...",[],fake,boatos.org,https://www.boatos.org/religiao/pastor-sergio-...,ps.: esse artigo e uma sugestao de leitores do...
freq,1,1,167,168,1,1,455,5201,5173,1,168


In [25]:
#Tratando os valores NaN com substituição por string vazia
df['message_norm_treatment'] = df['message_norm'].fillna("")

In [26]:
df=df.reset_index()

### Limpeza inicial do dataset

In [27]:
#Tratando os valores NaN com substituição por string vazia
df['message_norm_treatment'] = df['message_norm'].fillna("")

In [28]:
import re
# Procura as linhas com o padrão de telefone
df['telefone_match'] = df.message_norm_treatment.apply(lambda x: re.search('(((\(?\d{2}\)?\s)|(\(?\d{2}\)?\s))?((\d{4,5}\-\d{4})|(\d{4,5}\ \d{4}))|((\d{4,5}\d{5})|(\d{4,5}\ \d{4}))|(\(?\d{2}\)?\s)?(\d{4,5}\d{4})|((\(?\d{2}\)?\s)?(9.*))|(9\d{8})|\(61\))', x))
print('Quantidade de linhas com telefone encontradas:', df['telefone_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub('(((\(?\d{2}\)?\s)|(\(?\d{2}\)?\s))?((\d{4,5}\-\d{4})|(\d{4,5}\ \d{4}))|((\d{4,5}\d{5})|(\d{4,5}\ \d{4}))|(\(?\d{2}\)?\s)?(\d{4,5}\d{4})|((\(?\d{2}\)?\s)?(9.*))|(9\d{8})|\(61\))', '', x))
df['telefone_match'] = df.message_norm_treatment.apply(lambda x: re.search('(((\(?\d{2}\)?\s)|(\(?\d{2}\)?\s))?((\d{4,5}\-\d{4})|(\d{4,5}\ \d{4}))|((\d{4,5}\d{5})|(\d{4,5}\ \d{4}))|(\(?\d{2}\)?\s)?(\d{4,5}\d{4})|((\(?\d{2}\)?\s)?(9.*))|(9\d{8})|\(61\))', x))
print('Quantidade de linhas com telefone encontradas:', df['telefone_match'].count())
# Procura as linhas com o padrão de "artigos de sugestão"
df['sugestao_match'] = df.message_norm_treatment.apply(lambda x: re.search('(ps: esse artigo|ps:esse artigo|ps: |esse artigo foi|este artigo foi|esse artigo e|este artigo e|esse texto foi|ps\.\:).+((telefone |telefone -|facebook|numero |whatsapp)\.|(telefone   |telefone  |telefone |facebook|facebook |perfis: |whatsapp |6td |beso |facebook\. ))', x))
print('Quantidade de linhas com sugestões encontradas:', df['sugestao_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub('(ps: esse artigo|ps:esse artigo|ps: |esse artigo foi|este artigo foi|esse artigo e|este artigo e|esse texto foi|ps\.\:).+((telefone |telefone -|facebook|numero |whatsapp)\.|(telefone   |telefone  |telefone |facebook|facebook |perfis: |whatsapp |6td |beso |facebook\. ))', '', x))
df['sugestao_match'] = df.message_norm_treatment.apply(lambda x: re.search('(ps: esse artigo|ps:esse artigo|ps: |esse artigo foi|este artigo foi|esse artigo e|este artigo e|esse texto foi|ps\.\:).+((telefone |telefone -|facebook|numero |whatsapp)\.|(telefone   |telefone  |telefone |facebook|facebook |perfis: |whatsapp |6td |beso |facebook\. ))', x))
print('Quantidade de linhas com sugestões encontradas:', df['sugestao_match'].count())

Quantidade de linhas com telefone encontradas: 2158
Quantidade de linhas com telefone encontradas: 0
Quantidade de linhas com sugestões encontradas: 1500
Quantidade de linhas com sugestões encontradas: 0


In [29]:
#Substituindo palavra "boato" que indicia falsidade da notícia por string vazia
contador1 = 0
linha = 0
info_desnec_1 = 'boato '
for row in df["message_norm_treatment"]:
    if(info_desnec_1 not in row):
        linha+=1
        continue
    else:
        df["message_norm_treatment"][linha] = df["message_norm_treatment"][linha].replace(info_desnec_1, "")
        contador1+=1
        linha+=1
print("Um total de", contador1, "ocorrências da palavra", info_desnec_1,"foram substituídas por string vazia nos textos normalizados da notícia.")

Um total de 20 ocorrências da palavra boato  foram substituídas por string vazia nos textos normalizados da notícia.


In [30]:
#Substituindo "boatos.org" que indicia falsidade da notícia por string vazia
contador1 = 0
linha = 0
info_desnec_1 = 'boatos.org'
for row in df["message_norm_treatment"]:
    if(info_desnec_1 not in row):
        linha+=1
        continue
    else:
        df["message_norm_treatment"][linha] = df["message_norm_treatment"][linha].replace(info_desnec_1, "")
        contador1+=1
        linha+=1
print("Um total de", contador1, "ocorrências da palavra", info_desnec_1,"foram substituídas por string vazia nos textos normalizados da notícia.")

Um total de 15 ocorrências da palavra boatos.org foram substituídas por string vazia nos textos normalizados da notícia.


In [31]:
#NOVO PADRÃO ENCONTRADO E TRATADO
lista_linhas = []
padrao = 'edgard matsuki e editor'
for i in range(0, len(df)):
  if(padrao in df['message_norm_treatment'][i]):
    lista_linhas.append(i)
for e in lista_linhas:
  print(df['message_norm_treatment'][e])
  df['message_norm_treatment'][e] = ""

edgard matsuki e editor do site , site que ja desmentiu quase 6 mil noticias falsasuma das novidades do  para 2021 e a secao a semana em fakes. periodicamente, faremos analises sobre os assuntos mais recorrentes em termos de desinformacao na internet. este conteudo ficara aberto para republicacao em outros veiculos de midia (caso tenha interesse, entre em contato com o  para saber as condicoes). para ver todos os textos da secao, clique aqui.
edgard matsuki e editor do site , site que ja desmentiu quase 6 mil noticias falsasuma das novidades do  para 2021 e a secao a semana em fakes. periodicamente, faremos analises sobre os assuntos mais recorrentes em termos de desinformacao na internet. este conteudo ficara aberto para republicacao em outros veiculos de midia (caso tenha interesse, entre em contato com o  para saber as condicoes). para ver todos os textos da secao, clique aqui.
edgard matsuki e editor do site , site que ja desmentiu mais de 6 mil noticias falsasuma das novidades do 

In [32]:
#NOVO PADRÃO ENCONTRADO E TRATADO
contador = 0
lista_linhas = []
for row in df['message_norm_treatment']:
  if(row == ' e whatsapp no telefone -.'):
    print(row)
    lista_linhas.append(contador)
  contador+=1
for e in lista_linhas:
  df['message_norm_treatment'][e]=''

In [33]:
#NOVO PADRÃO ENCONTRADO E TRATADO
contador = 0
lista_linhas = []
for row in df['message_norm_treatment']:
  if(row == ' e whatsapp no telefone '):
    print(row)
    lista_linhas.append(contador)
  contador+=1
for e in lista_linhas:
  df['message_norm_treatment'][e]=''

 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 
 e whatsapp no telefone 


In [34]:
#ALTERAÇÃO 1 (NÃO É NECESSARIO TIRAR ISTO DO DATASET)
info = 'sortea'
for i in range(0, len(df['message_norm_treatment'])):
  if(info in df['message_norm_treatment'][i]):
    print(df['message_norm_treatment'][i],'\n')
    print(df['url_review'][i])
    print('\n')

amigos estou muito,muito,muito feliz,pois fui sorteado e ganhei 30 mil reais na promocao copa do mundo premiada cielo 2014 e estou compartilhando com voces para partiparem tambem dessa promocao,basta ter cartao de credito!muitas pessoas falam que nunca tem sorte de ganhar,mais tambem nao custa nada tentar ,sao apenas 2 minutos para participar da promocao.funciona assim , voce se cadastra no site , ganha uma numeracao , com essa numeracao voce vai participar da promocao.domingo dia 18/05 no programa do rodrigo faro na rede record , ira acontecer o sorteio , fique atento que voce pode ser sorteado como eu. bom , minha parte eu estou fazendo e compartilhando com voces , boa sorte a todos e estou torcendo muito por voces. acima vai o link pra voces fazerem o cadastroquem tem um pouquinho de discernimento ja deve saber que essa historia e mais um daqueles virus lindos e maravilhosos do facebook. como ja desmitificamos aqui alguns: virus da cor do facebook, ganhe dinheiro sem sair de casa, g

In [35]:
#Transformando os elementos da coluna 'tokens' em lista
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
for i in range(0, len(df)):
    df['tokens'][i] = tokenizer.tokenize(str(df['tokens'][i]))

In [36]:
#ALTERAÇÃO 2 NÃO É NECESSÁRIO TIRAR ISSO DO DATASET (APENAS LINHAS ESPECÍFICAS DESCRITAS NA CÉLULA ABAIXO)
info = 'siga'
contador = 0
for row in df['message_norm_treatment']:
  if(info in row):
    print(row)
    print(contador)
    print('\n')
  contador+=1

o filme malevola, que narra o conto da bela adormecida a partir da perspectiva da vila, vem sendo criticado por liderancas cristas, que aconselham pais cristaos a nao permitirem que seus filhos o assistam.sob a perspectiva religiosa, o colunista do wnd, drew zahn publicou um artigo afirmando que nao e aconselhavel assistir ao filme, e que o titulo em ingles, maleficent, significa fazer o mal, causar dano.zahn diz que no filme, a bruxa e retratada como alguem que, na verdade comecou como uma linda menina, o maior e mais belo dos seres celestiais, ate que o rei mal nesta historia tirou suas asas e lancou-a para a terra.a partir desse ponto, consumida pelo desejo de vinganca, ela se torna uma rainha paga sobre o mundo sobrenatural, auxiliada por seus criados, incluindo balthazar (o ultimo rei da babilonia, de acordo com o livro de daniel na biblia) e diablo, e torna seu objetivo de vida vingar-se do rei amaldicoando e destruindo seu filho. a semelhanca da historia de malevola com a narrat

In [37]:
#Substituição das informações desncessárias por string vazia
lista_nao_noticia = [2314, 2914, 2919, 3017, 3058, 4517, 4180, 4213, 4228, 4235]
for e in lista_nao_noticia:
  df['message_norm_treatment'][e] = ""

In [38]:
#ALTERAÇÃO 3 NÃO É NECESSÁRIO TIRAR ISSO DO DATASET
info = 'questionario'
contador = 0
lista = []
for row in df['message_norm_treatment']:
  if(info in row):
    print(row)
    print(contador)
    print('\n')
    lista.append(contador)
  contador+=1

assai atacadista contrata com urgencia, nao requer experiencia, inicio imediato *salario de ate: r$1.852,66* *vagas disponiveis em:*rede mercadao atacadista contrata com urgencia, nao requer experiencia, inicio imediato *salario de ate: r$1.320,81* *vagas disponiveis em:*estamos contratando com urgencia! nao precisa ter experiencia, inicio imediato. responda o questionario abaixo para liberarmos sua entrevista. pergunta 1 de 3: voce conhece o assai (mercadao atacadista) da sua cidade?
237


ola, o facebook selecionou milhares de perfis para responder um breve questionario e ganhar premios instantaneos, consegui acerta as respostas e ja recebi o meu presente tenta a sorte voce tambem.
1517


caixa economica informa que r$ 747 milhoes de impostos foram liberados para serem devolvido ao povo brasileiro* acesse o site e faca seu agendamento do saque [site] *eu acabei de pegar 3 mil reais!*o governo federal ira devolver mais de r$ 747 milhoes para todos os cidadaos, mesmo para aqueles que n

In [39]:
# Procura as linhas com o padrão de questionário
df['questionario_questao_match'] = df.message_norm_treatment.apply(lambda x: re.search(' questao 1 de 4:\w*', x))
print('Quantidade de linhas com boato encontradas:', df['questionario_questao_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub(' questao 1 de 4:\w*', '', x))
df['questionario_questao_match'] = df.message_norm_treatment.apply(lambda x: re.search(' questao 1 de 4:\w*', x))
print('Quantidade de linhas com questões encontradas:', df['questionario_questao_match'].count())

Quantidade de linhas com boato encontradas: 12
Quantidade de linhas com questões encontradas: 0


In [40]:
#ALTERAÇÃO 3 NÃO É NECESSÁRIO TIRAR ISSO DO DATASET
info = 'promocao'
for row in df['message_norm_treatment']:
  if(info in row):
    print(row)
    print('\n')

amigos estou muito,muito,muito feliz,pois fui sorteado e ganhei 30 mil reais na promocao copa do mundo premiada cielo 2014 e estou compartilhando com voces para partiparem tambem dessa promocao,basta ter cartao de credito!muitas pessoas falam que nunca tem sorte de ganhar,mais tambem nao custa nada tentar ,sao apenas 2 minutos para participar da promocao.funciona assim , voce se cadastra no site , ganha uma numeracao , com essa numeracao voce vai participar da promocao.domingo dia 18/05 no programa do rodrigo faro na rede record , ira acontecer o sorteio , fique atento que voce pode ser sorteado como eu. bom , minha parte eu estou fazendo e compartilhando com voces , boa sorte a todos e estou torcendo muito por voces. acima vai o link pra voces fazerem o cadastroquem tem um pouquinho de discernimento ja deve saber que essa historia e mais um daqueles virus lindos e maravilhosos do facebook. como ja desmitificamos aqui alguns: virus da cor do facebook, ganhe dinheiro sem sair de casa, g

In [41]:
#Buscando padrão "ps 2:" nas notícias e substituindo por string vazia
df['ps2_match'] = df.message_norm_treatment.apply(lambda x: re.search('(ps2:.*)', x))
print('Quantidade de linhas com ps2 encontradas:', df['ps2_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub('(ps2:.*)', '', x))
df['ps2_match'] = df.message_norm_treatment.apply(lambda x: re.search('(ps2:.*)', x))
print('Quantidade de linhas com ps2 encontradas:', df['ps2_match'].count())

Quantidade de linhas com ps2 encontradas: 50
Quantidade de linhas com ps2 encontradas: 0


In [42]:
# Procura as linhas com o padrão de palavras iniciando com boato
df['boato_match'] = df.message_norm_treatment.apply(lambda x: re.search('boato\w*', x))
print('Quantidade de linhas com boato encontradas:', df['boato_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub('boato\w*', '', x))
df['boato_match'] = df.message_norm_treatment.apply(lambda x: re.search('boato\w*', x))
print('Quantidade de linhas com boato encontradas:', df['boato_match'].count())

Quantidade de linhas com boato encontradas: 111
Quantidade de linhas com boato encontradas: 0


In [43]:
# Procura as linhas com o padrão de texto com links do bit.ly
df['bitly1_match'] = df.message_norm_treatment.apply(lambda x: re.search('http:\/\/bit\.ly\/.*whatsapp$', x))
print('Quantidade de linhas com links do bit.ly encontradas:', df['bitly1_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub('http:\/\/bit\.ly\/.*whatsapp$', '', x))
df['bitly1_match'] = df.message_norm_treatment.apply(lambda x: re.search('http:\/\/bit\.ly\/.*whatsapp$', x))
print('Quantidade de linhas com links do bit.ly encontradas:', df['bitly1_match'].count())

Quantidade de linhas com links do bit.ly encontradas: 0
Quantidade de linhas com links do bit.ly encontradas: 0


In [44]:
# Procura as linhas com o padrão de texto: "clique nos links bit.ly"
df['bitly2_match'] = df.message_norm_treatment.apply(lambda x: re.search('^clique nos links bit\.ly.*telegram$', x))
print('Quantidade de linhas com o padrão de texto: "clique nos links bit.ly" encontradas:', df['bitly2_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub('^clique nos links bit\.ly.*telegram$', '', x))
df['bitly2_match'] = df.message_norm_treatment.apply(lambda x: re.search('^clique nos links bit\.ly.*telegram$', x))
print('Quantidade de linhas com o padrão de texto: "clique nos links bit.ly" encontradas:', df['bitly2_match'].count())

Quantidade de linhas com o padrão de texto: "clique nos links bit.ly" encontradas: 0
Quantidade de linhas com o padrão de texto: "clique nos links bit.ly" encontradas: 0


In [45]:
# Procura as linhas com o padrão de texto iniciando e terminando com links bit.ly
df['bitly3_match'] = df.message_norm_treatment.apply(lambda x: re.search('^http:\/\/bit\.ly\/.*http:\/\/bit\.ly\/\w{7}$', x))
print('Quantidade de linhas com o padrão de texto iniciando e terminando com links bit.ly encontradas:', df['bitly3_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub('^http:\/\/bit\.ly\/.*http:\/\/bit\.ly\/\w{7}$', '', x))
df['bitly3_match'] = df.message_norm_treatment.apply(lambda x: re.search('^http:\/\/bit\.ly\/.*http:\/\/bit\.ly\/\w{7}$', x))
print('Quantidade de linhas com o padrão de texto iniciando e terminando com links bit.ly encontradas:', df['bitly3_match'].count())

Quantidade de linhas com o padrão de texto iniciando e terminando com links bit.ly encontradas: 0
Quantidade de linhas com o padrão de texto iniciando e terminando com links bit.ly encontradas: 0


In [46]:
#Removendo padrão link uber
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub(' > http://www.uber2anos.vai.la <  ', '', x))

### Limpeza final usando dataset sem notícias repetidas



In [47]:
#Definindo dataset auxiliar sem as notícias repetidas que será usado para achar novos padrões de conteúdo sem utilidade para o trabalho.

In [48]:
df1 = df.drop_duplicates(subset=['message', 'message_norm_treatment'])

In [49]:
df1.head()

,index,title,title_norm,message,message_norm,tokens,features,entities,type,source,url_review,message_norm_treatment,telefone_match,sugestao_match,questionario_questao_match,ps2_match,boato_match,bitly1_match,bitly2_match,bitly3_match
0,0,"Pastor Sérgio Von Helder, que chutou santa, nã...","pastor sergio von helder, que chutou santa, na...",NaN,NaN,"[pastor, sergio, von, helder, que, chutou, san...","{'len': 12, 1: {'token': 'pastor', 'lemma': 'p...",[Pastor Sérgio Von Helder],fake,boatos.org,https://www.boatos.org/religiao/pastor-sergio-...,,None,None,None,None,None,None,None,None
2,2,Lulinha não comprou Fazenda Fortaleza por 47 m...,lulinha nao comprou fazenda fortaleza por 47 m...,O Novo Mega Campeão do Brasil de enriqueciment...,o novo mega campeao do brasil de enriqueciment...,"[lulinha, nao, comprou, fazenda, fortaleza, po...","{'len': 8, 1: {'token': 'lulinha', 'lemma': 'l...","[Lulinha, Fazenda]",fake,boatos.org,https://www.boatos.org/politica/lulinha-fazend...,o novo mega campeao do brasil de enriqueciment...,None,None,None,None,None,None,None,None
3,3,Fanta Uva causa câncer e problema nos rins,fanta uva causa cancer e problema nos rins,ALERTA GERAL ATENÇÃO” FANTA UVA!!!! A propagan...,alerta geral atencao fanta uva!!!!a propaganda...,"[fanta, uva, causa, cancer, e, problema, nos, ...","{'len': 8, 1: {'token': 'fanta', 'lemma': 'fan...",[Fanta Uva],fake,boatos.org,https://www.boatos.org/saude/fanta-uva-da-canc...,alerta geral atencao fanta uva!!!!a propaganda...,None,None,None,None,None,None,None,None
8,8,Vírus em rede social: Facefollow - Ganhe segui...,virus em rede social: facefollow - ganhe segui...,Facefollow – Ganhe seguidores no FacebookAté 5...,facefollow ganhe seguidores no facebookate 50...,"[virus, em, rede, social, facefollow, ganhe, s...","{'len': 11, 1: {'token': 'virus', 'lemma': 'vi...","[Facefollow, Ganhe, Facebook]",fake,boatos.org,https://www.boatos.org/tecnologia/virus-em-red...,facefollow ganhe seguidores no facebookate 50...,None,None,None,None,None,None,None,None
9,9,Mensagem falsa aponta que WhatsApp vai cobrar ...,mensagem falsa aponta que whatsapp vai cobrar ...,PS: esse artigo foi uma sugestão do leitor (e ...,ps: esse artigo foi uma sugestao do leitor (e ...,"[mensagem, falsa, aponta, que, whatsapp, vai, ...","{'len': 9, 1: {'token': 'mensagem', 'lemma': '...",[Mensagem],fake,boatos.org,https://www.boatos.org/tecnologia/mensagem-fal...,,None,None,None,None,None,None,None,None


In [50]:
df1.shape

(4207, 20)

In [51]:
#Bloco de código com for nas linhas do novo dataset para ler as notícias sem repetições e identificar novos padrões

In [52]:
for row in df1['message_norm_treatment']:
    print(row, '\n')

 

o novo mega campeao do brasil de enriquecimento subito e o proprietario desta fazenda. fazenda fortaleza comprada e certificada em cartorio de registro de imoveis .proprietario: fabio luis lula da silva (isso mesmo)propriedade: fazenda na regiao de valparaiso/sppreco: 47 milhoes de reaispois e, o lulinha, o filho do presidente lula, que ha 05 anos era subempregado do zoologico em sao paulo, agora acabou de comprar a fazenda fortaleza (de porteira fechada) localizada as margens da rodovia marechal rondon, municipio de valparaiso-sp, de propriedade do sr. jose carlos prata cunha, um dos maiores produtores de boi nelore do brasil, pela simples bagatela de r$ 47.000.000,00 (quarenta e sete milhoes de reais).o impressionante nesta trajetoria de sucesso e rara inteligencia e que de um salario de r$ 1.500,00 passou a proprietario, de uma vez so, de um patrimonio de 47 milhoes. o que levaria 2.612 anos.veja so que competencia, logo depois de comprar, levou essa fazenda a ser a primeira a re




#### Temos que o dataset original possui 8517 linhas, já o novo dataset auxiliar com a repetição de notícias removidas tem 4209 linhas, o que indica grande quantidade de notícias repetidas. Com isto, usaremos o df1 para fazer consultas e identificar padrões, e posteriormente aplicaremos expressões regulares para tratar estes padrões do df original. Estas ações seguem abaixo:




In [53]:
info = '[]'
for row in df['message_norm_treatment']:
    if info in row:
        print(row, '\n')

sao paulo, 26 de julho de 2005carta n001 26/07/2005 em 1 viaeis que vi no ano de 2014, precisa-se em 26 de novembro, um aviao da [], chocar-se contra o edificio do []  cruzamento da avenida paulista com a alameda campinas, proximo ao maksoud plaza e podera matar centenas de pessoas. atenciosamente,prof. juscelino nobrega da luzdireito de resposta :a carta enviada em 2005 por duas vezes (sic) e registrada na epoca ao sr. eduardo campos , (sic) foi na primeira missiva (sic) mostrada na imagem da materia enviada ao entao , (sic) governador jarbas de andrade vasconcelos e que esse nome nao foi citado na primeira carta de 15/06/2005  e apenas preferiu colocar aos cuidados de eduardo campos  e nao como tenta induzir a materia relacionada  ao governador  e sim, na segunda missiva enviada apenas em setembro de 2005  jucelino nobrega da luz , (sic) referiu-se ao governador jarbas de andrade vasconcelos e inclusive envia novamente a copia da carta em referencia da materia aqui especificada  no c

In [54]:
info = 'print de noticia falsa'
lista_padrao = []
cont1=0
for row in df['message_norm_treatment']:
    if info in row:
        print(row, '\n')
        lista_padrao.append(cont1)
    cont1+=1

print de noticia falsa sobre iraniana que fez 50 plasticas para ficar igual angelina jolie  

print de noticia falsa que aponta que pedroo grandene ganhou us$ 3,5 milhoes no cassino ontem 

print de noticia falsa sobre meteoro em sinop 

print de noticia falsa que aponta que eduardo paes leva vida de luxo em nova york 

print de noticia falsa que aponta que silvio santos conheceu roque pedindo emprego em lanchonete. 

print de noticia falsa sobre o desaparecimento da menina yasmin 

print de noticia falsa que aponta que datena sera processado por causa de video sobre salario minimo. 

print de noticia falsa sobre 30 aniversario da samsung. 

print de noticia falsa sobre veiculos apreendidos na lava jato  

print de noticia falsa sobre camera com infravermelho em lombadas eletronicas em curitiba 

print de noticia falsa que aponta que beber agua em garrafa de plastico aquecida causa cancer de mama 

print de noticia falsa que aponta mamografia de r$ 10 reais na clinica bueno.  

print d

In [55]:
#Removendo palavra 'falsa' do padrão 'print de notícia falsa...' dos textos das notícias que denuncia a notícia ser fake.
info = 'falsa '
for i in range(0, len(lista_padrao)):
    df['message_norm_treatment'][lista_padrao[i]] = df['message_norm_treatment'][lista_padrao[i]].replace(info, '')

In [56]:
info = 'print da noticia falsa'
lista_padrao1 = []
cont1=0
for row in df['message_norm_treatment']:
    if info in row:
        print(row, '\n')
        lista_padrao1.append(cont1)
    cont1+=1

print da noticia falsa que aponta que aloysio nunes diz que brasil vai agir contra eua em caso de invasao da venezuela  

print da noticia falsa que aponta que pabllo vittar saira do brasil caso bolsonaro seja eleito.  

print da noticia falsa que aponta que lindbergh farias pede que general mourao seja levado algemado ao senado 



In [57]:
#Removendo palavra 'falsa' do padrão 'print da notícia falsa...' dos textos das notícias que denuncia a notícia ser fake.
info = 'falsa '
for i in range(0, len(lista_padrao1)):
    df['message_norm_treatment'][lista_padrao1[i]] = df['message_norm_treatment'][lista_padrao1[i]].replace(info, '')

In [58]:
info = 'edgard matsuki'
cont = 0
for row in df['message_norm_treatment']:
    if info in row:
        print(row, '\n')
        cont+=1
        print(cont)

ha alguns anos, entrei numa estacao de metro em estocolmo, a tao civilizada capital da tao primeiro-mundista suecia, e notei que havia entre muitas catracas comuns uma de passagem livre. questionei a vendedora de bilhetes o porque daquela catraca permanentemente liberada, sem nenhum seguranca por perto, e ela me explicou que era destinada as pessoas que por qualquer motivo nao tivessem dinheiro para a passagem. minha mente incredula e cheia de jeitinhos brasileiros nao conteve a pergunta obvia (para nos!): e se a pessoa tiver dinheiro mas simplesmente quiser burlar a lei?aqueles olhos suecos e azuis se espremeram num sorriso de pureza constrangedora  mas por que ela faria isso?, me perguntou. nao lhe respondi. comprei o bilhete, passei pela catraca e atras de mim uma multidao que tambem havia pago por seus bilhetes. a catraca livre continuava vazia, tao vazia quanto minha alma brasileira  e envergonhada. quem faz um pais e o povo!!!!!!  oi. meu nome e edgard matsuki e eu sou jornalista

In [59]:
#Removendo padrão 'edgard matsuki' dos textos das notícias que denuncia a notícia ser fake, visto que é o editor do site que verifica notícias falsas.
df['edgard_match'] = df.message_norm_treatment.apply(lambda x: re.search("(pergunta: meu nome e edgard matsuki.*|oi. meu nome e edgard matsuki.*|edgard matsuki e.*)", x))
print('Quantidade de linhas com o padrão "edgard matsuki" encontradas:', df['edgard_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub("(pergunta: meu nome e edgard matsuki.*|oi. meu nome e edgard matsuki.*|edgard matsuki e.*)", '', x))
df['edgard_match'] = df.message_norm_treatment.apply(lambda x: re.search("(pergunta: meu nome e edgard matsuki.*|oi. meu nome e edgard matsuki.*|edgard matsuki e.*)", x))
print('Quantidade de linhas com o padrão "edgard matsuki" encontradas:', df['edgard_match'].count())

Quantidade de linhas com o padrão "edgard matsuki" encontradas: 1
Quantidade de linhas com o padrão "edgard matsuki" encontradas: 0


In [60]:
info = 'confira a nossa nova secao oportunidades'
cont = 0
for row in df['message_norm_treatment']:
    if info in row:
        print(row, '\n')
        cont+=1
        print(cont) 

In [61]:
#Removendo padrão 'confira a nossa nova secao oportunidades clicando aqui...' dos textos das notícias.
df['confira_secao_match'] = df.message_norm_treatment.apply(lambda x: re.search("(ps2: confira a nossa nova secao oportunidades.*)", x))
print('Quantidade de linhas com o padrão "confira a nossa nova secao oportunidades clicando aqui..." encontradas:', df['confira_secao_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub("(ps2: confira a nossa nova secao oportunidades.*)", '', x))
df['confira_secao_match'] = df.message_norm_treatment.apply(lambda x: re.search("(ps2: confira a nossa nova secao oportunidades.*)", x))
print('Quantidade de linhas com o padrão "confira a nossa nova secao oportunidades clicando aqui..." encontradas:', df['confira_secao_match'].count())

Quantidade de linhas com o padrão "confira a nossa nova secao oportunidades clicando aqui..." encontradas: 0
Quantidade de linhas com o padrão "confira a nossa nova secao oportunidades clicando aqui..." encontradas: 0


In [62]:
info = 'http://bit.ly/2ou3zwz'
cont = 0
for row in df['message_norm_treatment']:
    if info in row:
        print(row, '\n')
        cont+=1
        print(cont) 

comprovado! sergio moro filiado ao psdb. http://bit.ly/2ou3zwz siga-nos no twitterhttp://bit.ly/2ot6bek siga-nos no youtubehttp://bit.ly/2ahn 

1
grupo chines compra 23,7% da azul por r$ 1,7 bi o acordo dara a azul mais tempo para fazer sua planejada abertura de capital, segundo o presidente-executivo da companhia, antonoaldo neves. nao estavamos com pressao pelo ipo (oferta inicial de acoes, na sigla em ingles), mas agora isso nos da um prazo ainda maior, disse ele.terceira maior empresa do setor aereo no brasil, a azul desistiu em junho, pela terceira vez, de fazer o ipo devido as condicoes adversas do mercado de capitais. a desistencia ocorreu pouco antes de a empresa receber um aporte de 100 milhoes de dolares da americana united airlines, do grupo united continental. []http://bit.ly/2ou3zwz siga-nos no twitterhttp://bit.ly/2ot6bek siga-nos no youtubehttp://bit.ly/2ahn 

2
empresario dono de rede de hoteis em nova york se suicidando depois de um grande prejuizo e ter tido que manda

In [63]:
#Removendo padrão com série de links para seguir páginas dos textos das notícias.
df['bitly_segue_match'] = df.message_norm_treatment.apply(lambda x: re.search("(http://bit.ly/2ou3zwz.*)", x))
print('Quantidade de linhas com o padrão com links bitly para seguir páginas encontradas:', df['bitly_segue_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub("(http://bit.ly/2ou3zwz.*)", '', x))
df['bitly_segue_match'] = df.message_norm_treatment.apply(lambda x: re.search("(http://bit.ly/2ou3zwz.*)", x))
print('Quantidade de linhas com o padrão com links bitly para seguir páginas encontradas:', df['bitly_segue_match'].count())

Quantidade de linhas com o padrão com links bitly para seguir páginas encontradas: 73
Quantidade de linhas com o padrão com links bitly para seguir páginas encontradas: 0


#### Novo tratamento necessário para remover padrão 'clique nos links bit.ly para acessar nossos perfis' que só ocorrem no fim das notícias

In [64]:
for i in range(0, len(df)):
    if('bit.ly' in df['message_norm_treatment'][i]):
        print(df['message_norm_treatment'][i],'\n')

clique nos links bit.ly para acessar nossos perfis: 

heranca maldita: mandetta renova contratos de publicidade de r$ 1bilhao firmados no governo dilma de forma silenciosa e sem o aval do planalto, o ministro da saude luiz henrique mandetta renovou contratos de publiciade que ultrapassam r$ 1 bilhao com agencias de publicidade que alimentam a midia contra o presidente jair bolsonaro. nao e a toa, que mandetta virou o ministro queridinho da extrema imprensa, pois o ministerio da saude escoa recursos para empresas de comunicacao como globo e band que de forma orquestrada firmaram parceria em novembro com a china media group, estatal de comunicacao do gigante asiatico  braco midiatico do partido comunista da china. []clique nos links bit.ly para acessar nossos perfis: 

esses peixes foram pescados em porto de galinhas e os pescadores nao venderam mas deram para a populacao pois entenderam que era o senhor que tinha enviado esta grande quantidade de peixes para alimentar o povo!clique nos 

In [65]:
info='clique nos links bit.ly para acessar nossos perfis'
for i in range(0, len(df)):
    if(info in df['message_norm_treatment'][i]):
        df['message_norm_treatment'][i] = df['message_norm_treatment'][i].replace(info, '')

#### Problema com 00 ou 000 sendo encontrados no most_common 

In [66]:
df['zeros'] = df.message_norm_treatment.apply(lambda x: re.search("(( 00 )|( 000 ))", x))
print('Quantidade de linhas com o padrão "00 000" encontradas:', df['zeros'].count())

Quantidade de linhas com o padrão "00 000" encontradas: 4


In [67]:
lista_linhas_zeros = []
for i in range(0, len(df)):
    if((df['zeros'][i])!= None):
        lista_linhas_zeros.append(i)
        print(df['message'][i], '\n')

LEIA ISTO : PROJETO DE EMENDA À CONSTITUIÇÃO DE INICIATIVA POPULAR ENTREGUE ONTEM AO CONGRESSO NACIONAL COM O NÚMERO RECORDE DE 2. 500. 000. 000 (DOIS MILHÕES E QUINHENTAS MIL ASSINATURAS!! MAIOR DO QUE O DA “FICHA LIMPA”! ) ORGULHOSAMENTE REPASSO.LUCIANA UCELLI. A cobra vai fumar. Em três dias, a maioria das pessoas no Brasil terá esta mensagem. Esta é uma idéia que realmente deve ser considerada e repassada para o Povo. Lei de Reforma do Congresso de 2013 (emenda à Constituição) PEC de iniciativa popular: Lei de Reforma do Congresso (proposta de emenda à Constituição Federal)…PS: Esse artigo foi uma sugestão de diversos leitores via WhatsApp. Se você quiser sugerir um tema para o Boatos.org, entre em contato com a gente pelo site, Facebook ou WhatsApp, no telefone (61) 99331 6821. 

‘Poupancinha do Operário. Descobertos US$ 108 Milhões de LULA em Luxemburgo.Luxemburgo (Luxemburg em alemão e Luxembourg em francês) é uma comuna de Luxemburgo com status de cidade, pertencente ao distrit

In [68]:
#Tratando espaçamento entre os 0's
for e in lista_linhas_zeros:
    df['message_norm_treatment'][e] = df['message_norm_treatment'][e].replace(r'. ', '.')

In [69]:
#Problema resolvido
for e in lista_linhas_zeros:
    print(df['message_norm_treatment'][e])

leia isto :projeto de emenda a constituicao de iniciativa popular entregue ontem ao congresso nacional com o numero recorde de 2.500.000.000 (dois milhoes e quinhentas mil assinaturas!! maior do que o da ficha limpa! ) orgulhosamente repasso.luciana ucelli.a cobra vai fumar.em tres dias, a maioria das pessoas no brasil tera esta mensagem.esta e uma ideia que realmente deve ser considerada e repassada para o povo.lei de reforma do congresso de 2013 (emenda a constituicao) pec de iniciativa popular: lei de reforma do congresso (proposta de emenda a constituicao federal)
poupancinha do operario.descobertos us$ 108 milhoes de lula em luxemburgo.luxemburgo (luxemburg em alemao e luxembourg em frances) e uma comuna de luxemburgo com status de cidade, pertencente ao distrito de luxemburgo e ao cantao de luxemburgo.luxemburgo e uma cidade muito desenvolvida em seu comercio e nas industrias.a area metropolitana do luxemburgo, que inclui hesperange, sandweiler, strassen e walferdange tem uma pop

In [70]:
# Procura as linhas com o padrão de texto com 00
df['zeros_espacados_match'] = df.message_norm_treatment.apply(lambda x: re.search("0\. 0|0 0", x))
print('Quantidade de linhas com zeros encontrados:', df['zeros_espacados_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub(r"(0\. 0|0 0)", "00", x))
df['zeros_espacados_match'] = df.message_norm_treatment.apply(lambda x: re.search("0\. 0|0 0", x))
print('Quantidade de linhas com zeros encontrados:', df['zeros_espacados_match'].count())

Quantidade de linhas com zeros encontrados: 1
Quantidade de linhas com zeros encontrados: 0


In [71]:
# Procura as linhas com o padrão de texto com .000 e tranforma em 000 (RESOLVE PROBLEMA DOS 3 0'S)
df['zeros_triplos_match'] = df.message_norm_treatment.apply(lambda x: re.search("(\.000)", x))
print('Quantidade de linhas com zeros encontrados:', df['zeros_triplos_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub(r"(\.000)", "000", x))
df['zeros_triplos_match'] = df.message_norm_treatment.apply(lambda x: re.search("(\.000)", x))
print('Quantidade de linhas com zeros encontrados:', df['zeros_triplos_match'].count())

Quantidade de linhas com zeros encontrados: 114
Quantidade de linhas com zeros encontrados: 0


In [72]:
#Problema resolvido (PODE SER QUE A TOKENIZAÇÃO CONSIDERA '.' COMO FIM DA PALAVRA)
for e in lista_linhas_zeros:
    print(df['message_norm_treatment'][e])

leia isto :projeto de emenda a constituicao de iniciativa popular entregue ontem ao congresso nacional com o numero recorde de 2.500000000 (dois milhoes e quinhentas mil assinaturas!! maior do que o da ficha limpa! ) orgulhosamente repasso.luciana ucelli.a cobra vai fumar.em tres dias, a maioria das pessoas no brasil tera esta mensagem.esta e uma ideia que realmente deve ser considerada e repassada para o povo.lei de reforma do congresso de 2013 (emenda a constituicao) pec de iniciativa popular: lei de reforma do congresso (proposta de emenda a constituicao federal)
poupancinha do operario.descobertos us$ 108 milhoes de lula em luxemburgo.luxemburgo (luxemburg em alemao e luxembourg em frances) e uma comuna de luxemburgo com status de cidade, pertencente ao distrito de luxemburgo e ao cantao de luxemburgo.luxemburgo e uma cidade muito desenvolvida em seu comercio e nas industrias.a area metropolitana do luxemburgo, que inclui hesperange, sandweiler, strassen e walferdange tem uma popul

In [73]:
#Identificando novo padrão
for i in range(0, len(df['message_norm_treatment'])-1):
    if(" abs" in df['message_norm_treatment'][i]):
        print(df['message_norm_treatment'][i])

ela nao precisa se preocupar comigo, estou muito bem cuidado pelo povo brasileiro e por medicos muito capacitados. porque ela nao fez nenhuma referencia ao desastre de bh?a preocupacao real dela e que se o brasil perder a copa, ela perde as eleicoes, mas se deus existe e ele existe, ela perdera as eleicoes e o brasil ganhara a copa. #forcabrasil #forcabh. todo meu respeito e solidariedade aos familiares do desastre em bh!esta montagem e um absurdo e nao existe. o neymar nao disse essas palavras! tudo o que o neymar diz e as formas de se expressar oficialmente sao atraves de suas plataformas oficiais
pegue um pedaco de dentro da casca de banana e esfregue suavemente em torno dos seus dentes por cerca de 2 minutos. os minerais surpreendentes na casca como o potassio, magnesio e manganes, absorvem em seus dentes e em seguida dentes brancos.
na madrugada desta quinta-feira (14), um dos grandes absurdos ocorrido no julgamento do traficante luiz fernando da costa, o fernandinho beira-mar, cu

In [74]:
#Identificando novo padrão
for i in range(0, len(df['message_norm_treatment'])-1):
    if(" *repassando*" in df['message_norm_treatment'][i]):
        print(df['message_norm_treatment'][i])

emprego. .. a andrade e gutierrez construtora vai contratar 150 profissionais para trabalhar na duplicacao da rodovia br 040 interessados entrar em contato para processo seletivo: whatsapp ate a data. marilene ou eliane *gerente de rh*, vigilante, porteiro, motoboy, copeira, secretaria, auxiliar administrativo, auxiliar de escritorio, auxiliar de producao, auxiliar logistica, auxiliar de almoxarifado, operador de escavadeiras hidraulicas, operador de motoniveladora, operador de pa carregadeira, operador de trator de esteira, tecnico em geologia, tecnico em seguranca do trabalho, tecnico em enfermagem, tecnico em mecanica, mecanico industrial, eletricista, eletronico, eletrotecnico, soldador e marceneiro.oferece : salario basico em torno de r$ 2.700,00 ate r$ 5.500,00, mais tiquete r$ 620,00 -plano de saude. talvez voce pode nao estar precisando, mas pode ter um amigo pai de familia desempregado. divulgue. *repassando* 
pessoal novo golpe perigoso: recebi alguns telefonemas hoje do meu 

In [75]:
# Procura as linhas com o padrão de texto 'repassando de whatsapp'
df['repassando_match'] = df.message_norm_treatment.apply(lambda x: re.search("(( abs | \*repassando*).*)", x))
print('Quantidade de linhas com texto repassando encontrados:', df['repassando_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub("(( abs| \*repassando*).*)", "", x))
df['repassando_match'] = df.message_norm_treatment.apply(lambda x: re.search("(( abs| \*repassando*).*)", x))
print('Quantidade de linhas com texto repassando encontrados:', df['repassando_match'].count())

Quantidade de linhas com texto repassando encontrados: 3
Quantidade de linhas com texto repassando encontrados: 0


In [76]:
# Procura as linhas com o padrão ,00 para remover (RESOLVE O PROBLEMA DOS 00'S)
df['virgulazerozero_match'] = df.message_norm_treatment.apply(lambda x: re.search(",00", x))
print('Quantidade de linhas com ,00 encontrados:', df['virgulazerozero_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub(",00", "", x))
df['virgulazerozero_match'] = df.message_norm_treatment.apply(lambda x: re.search(",00", x))
print('Quantidade de linhas com ,00 encontrados:', df['virgulazerozero_match'].count())

Quantidade de linhas com ,00 encontrados: 116
Quantidade de linhas com ,00 encontrados: 0


In [77]:
#Após tratamento removendo ,00 (RESOLVEU A OCORRÊNCIA DE 00 NOS TÓPICOS (PODE SER QUE A TOKENIZAÇÃO CONSIDERA APÓS VÍRGULA COMO OUTRA PALAVRA))
for i in range(0, len(df['message_norm_treatment'])-1):
    if("experiencia" in df['message_norm_treatment'][i]):
        print(df['message_norm_treatment'][i])

coca-cola com *81* vagas disponiveis, nao precisa ter experiencia, agende sua entrevista!*salario a partir de:* r$2.147,52 mais todos os beneficios.*vagas disponiveis em:*http://empregos.clubdepois nao diga que nao teve oportunidade
decretado hoje: venezuela e hoje uma nova cuba : deus tenha misericordia do povo venezuelano e livre o brasil desse risco! urgente  so algumas das 44 novas medidas da venezuela ordenadas por maduro, ontem 03/08/17 (vao vendo o horror): os bancos particulares serao todos estatais   criacao do sistema nacional bolivariano   o salario minimo sera revisado para todos que recebem do estado. os trabalhadores tecnicos e profissionais receberao todos por igual, com base no salario minimo revisado   se incrementa a escala de bonus de acordo com experiencia   todos cartoes de credito serao eliminados, somente alguns trabalhadores do estado terao seus cartoes mantidos  prisao para quem tiver qualquer moeda estrangeira no territorio nacional   proibidas contas bancaria

In [78]:
#Padrão (11) encontrado
for i in range(0, len(df['message_norm_treatment'])-1):
    if("(11)" in df['message_norm_treatment'][i]):
        print(df['message_norm_treatment'][i])   

este numero de celular clona o seu numero !!!repassandonumero de celular que clona o numero chamadopasse para o maior numero de pessoas.nao atenda ligacao do celular de numero:*(11)
uma das residencias do ex. presidente luis inacio lula da silva foi assaltada na manha desta sexta-feira (11), no bairro presidente medici, em chapeco. o fato ocorreu na rua montevideu.segundo a policia miltiar (pm), no primeiro momento, a guarnicao foi acionada para atender ocorrencia de furto e se deparou com o assalto.chegando ao local, a guarnicao visualizou as portas de frente da residencia arrombadas e em contato com a vitima, uma mulher de 32 anos, foi constatado que se tratava de um assalto. os suspeitos entraram no local e ao se deparar com a vitima, que acordou no momento da acao, ameacaram a mesma, inclusive, os suspeitos disseram que iriam atirar caso a vitima reagisse. a mulher foi trancada no quarto com seus filhos e em seguida, os suspeitos fugiram com alguns objetos.objetos:televisor led 57,

In [79]:
# Procura as linhas com o padrão (11) para remover
df['padrao11_match'] = df.message_norm_treatment.apply(lambda x: re.search("((\:\*\(11\))|( interessados enviar curriculum para: \[emailprotected\] ou ligar no\(11\)))", x))
print('Quantidade de linhas com (11) encontrados:', df['padrao11_match'].count())
df['message_norm_treatment'] = df.message_norm_treatment.apply(lambda x: re.sub("((\:\*\(11\))|( interessados enviar curriculum para: \[emailprotected\] ou ligar no\(11\)))", "", x))
df['padrao11_match'] = df.message_norm_treatment.apply(lambda x: re.search("((\:\*\(11\))|( interessados enviar curriculum para: \[emailprotected\] ou ligar no\(11\)))", x))
print('Quantidade de linhas com (11) encontrados:', df['padrao11_match'].count())

Quantidade de linhas com (11) encontrados: 2
Quantidade de linhas com (11) encontrados: 0


## Criação da coluna 'message_norm_treatment_ssw' que não possui stopwords básicas

In [80]:
# Criando coluna com o texto das notícias normalizadas que será usada para o tratamento e remoção das stopwords
df['message_norm_treatment_ssw'] = df['message_norm_treatment']

In [81]:
#Tokenizando coluna para remover as stopwords
contador3 = 0
for row in df['message_norm_treatment_ssw']:
  df['message_norm_treatment_ssw'][contador3] = word_tokenize(row)
  contador3+=1

In [82]:
#Definindo idioma para recuperação das stopwords
stops = stopwords.words('portuguese')

In [83]:
# Transformando punctuation em uma lista, e inserindo na lista das stops_words
punctuation = list(punctuation)
# Juntando as duas listas em uma só
stops.extend(punctuation)

In [84]:
#Removendo as stopwords na coluna que tem a notícia tokenizada
contador4 = 0
for row in df['message_norm_treatment_ssw']:
  df['message_norm_treatment_ssw'][contador4] = [word for word in row if word not in stops]
  contador4+=1

## Criação da coluna 'message_norm_treatment_ssw2' que não possui stopwords avançadas

In [85]:
#Criando coluna 'message_norm_treatment_ssw2' que não possui stopwords mais avançadas
df['message_norm_treatment_ssw2'] = df['message_norm_treatment_ssw']

In [86]:
from urllib.request import urlopen
advanced_sw_list = []
file = open("filename","w", encoding='utf-8')
url = urlopen("https://gist.githubusercontent.com/alopes/5358189/raw/2107d809cca6b83ce3d8e04dbd9463283025284f/stopwords.txt")
for line in url:
    file.write(str(line) +'\n')
    advanced_sw_list.append(line)
file.close()

In [87]:
#Removendo espaços e \n dos elementos da lista avançada de strings(stopwords)
advanced_sw_list1 = list(map(lambda x:x.strip(),advanced_sw_list))

In [88]:
#Decode para remover o b frente das strings(stopwords) 
for i in range(0, len(advanced_sw_list1)):
    advanced_sw_list1[i] = advanced_sw_list1[i].decode('utf-8')

In [89]:
!pip install unidecode

In [90]:
#Removendo acentuação das strings(stopwords)
import unidecode 
for i in range(0, len(advanced_sw_list1)):
    advanced_sw_list1[i] = unidecode.unidecode(advanced_sw_list1[i])

In [91]:
#Adicionando novas stopwords manualmente
advanced_sw_list1.append('ai')
advanced_sw_list1.append('sobre')
advanced_sw_list1.append('r')
advanced_sw_list1.append('pra')
advanced_sw_list1.append('entao')
advanced_sw_list1.append('ta')
advanced_sw_list1.append('porque')
advanced_sw_list1.append('assim')
advanced_sw_list1.append('onde')
advanced_sw_list1.append('aqui')
advanced_sw_list1.append('agora')
advanced_sw_list1.append('vai')
advanced_sw_list1.append('ainda')
advanced_sw_list1.append('vamos')
advanced_sw_list1.append('diz')
advanced_sw_list1.append('pode')
advanced_sw_list1.append('sendo')
advanced_sw_list1.append('entao')
advanced_sw_list1.append('aqui')
advanced_sw_list1.append('1')

In [92]:
#Mais stopwords
mais_stopwords = "a, agora, ainda, alguém, algum, alguma, algumas, alguns, ampla, amplas, amplo, amplos, ante, antes, ao, aos, após, aquela, aquelas, aquele, aqueles, aquilo, as, até, através, cada, coisa, coisas, com, como, contra, contudo, da, daquele, daqueles, das, de, dela, delas, dele, deles, depois, dessa, dessas, desse, desses, desta, destas, deste, deste, destes, deve, devem, devendo, dever, deverá, deverão, deveria, deveriam, devia, deviam, disse, disso, disto, dito, diz, dizem, do, dos, e, é, ela, elas, ele, eles, em, enquanto, entre, era, essa, essas, esse, esses, esta, está, estamos, estão, estas, estava, estavam, estávamos, este, estes, estou, eu, fazendo, fazer, feita, feitas, feito, feitos, foi, for, foram, fosse, fossem, grande, grandes, há, isso, isto, já, la, lá, lhe, lhes, lo, mas, me, mesma, mesmas, mesmo, mesmos, meu, meus, minha, minhas, muita, muitas, muito, muitos, na, não, nas, nem, nenhum, nessa, nessas, nesta, nestas, ninguém, no, nos, nós, nossa, nossas, nosso, nossos, num, numa, nunca, o, os, ou, outra, outras, outro, outros, para, pela, pelas, pelo, pelos, pequena, pequenas, pequeno, pequenos, per, perante, pode, pude, podendo, poder, poderia, poderiam, podia, podiam, pois, por, porém, porque, posso, pouca, poucas, pouco, poucos, primeiro, primeiros, própria, próprias, próprio, próprios, quais, qual, quando, quanto, quantos, que, quem, são, se, seja, sejam, sem, sempre, sendo, será, serão, seu, seus, si, sido, só, sob, sobre, sua, suas, talvez, também, tampouco, te, tem, tendo, tenha, ter, teu, teus, ti, tido, tinha, tinham, toda, todas, todavia, todo, todos, tu, tua, tuas, tudo, última, últimas, último, últimos, um, uma, umas, uns, vendo, ver, vez, vindo, vir, vos, vós"
mais_stopwords1 = [ 'a', 'à', 'adeus', 'agora', 'aí', 'ainda', 'além', 'algo', 'alguém', 'algum', 'alguma', 'algumas', 'alguns', 'ali', 'ampla', 'amplas', 'amplo', 'amplos', 'ano', 'anos', 'ante', 'antes', 'ao', 'aos', 'apenas', 'apoio', 'após', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aqui', 'aquilo', 'área', 'as', 'às', 'assim', 'até', 'atrás', 'através', 'baixo', 'bastante', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cá', 'cada', 'catorze', 'cedo', 'cento', 'certamente', 'certeza', 'cima', 'cinco', 'coisa', 'coisas', 'com', 'como', 'conselho', 'contra', 'contudo', 'custa', 'da', 'dá', 'dão', 'daquela', 'daquelas', 'daquele', 'daqueles', 'dar', 'das', 'de', 'debaixo', 'dela', 'delas', 'dele', 'deles', 'demais', 'dentro', 'depois', 'desde', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'deve', 'devem', 'devendo', 'dever', 'deverá', 'deverão', 'deveria', 'deveriam', 'devia', 'deviam', 'dez', 'dezanove', 'dezasseis', 'dezassete', 'dezoito', 'dia', 'diante', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'dizer', 'do', 'dois', 'dos', 'doze', 'duas', 'dúvida', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'embora', 'enquanto', 'entre', 'era', 'eram', 'éramos', 'és', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estás', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estiveste', 'estivestes', 'estou', 'etc', 'eu', 'exemplo', 'faço', 'falta', 'favor', 'faz', 'fazeis', 'fazem', 'fazemos', 'fazendo', 'fazer', 'fazes', 'feita', 'feitas', 'feito', 'feitos', 'fez', 'fim', 'final', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'forma', 'formos', 'fosse', 'fossem', 'fôssemos', 'foste', 'fostes', 'fui', 'geral', 'grande', 'grandes', 'grupo', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'havia', 'hei', 'hoje', 'hora', 'horas', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'la', 'lá', 'lado', 'lhe', 'lhes', 'lo', 'local', 'logo', 'longe', 'lugar', 'maior', 'maioria', 'mais', 'mal', 'mas', 'máximo', 'me', 'meio', 'menor', 'menos', 'mês', 'meses', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'mil', 'minha', 'minhas', 'momento', 'muita', 'muitas', 'muito', 'muitos', 'na', 'nada', 'não', 'naquela', 'naquelas', 'naquele', 'naqueles', 'nas', 'nem', 'nenhum', 'nenhuma', 'nessa', 'nessas', 'nesse', 'nesses', 'nesta', 'nestas', 'neste', 'nestes', 'ninguém', 'nível', 'no', 'noite', 'nome', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'nova', 'novas', 'nove', 'novo', 'novos', 'num', 'numa', 'número', 'nunca', 'o', 'obra', 'obrigada', 'obrigado', 'oitava', 'oitavo', 'oito', 'onde', 'ontem', 'onze', 'os', 'ou', 'outra', 'outras', 'outro', 'outros', 'para', 'parece', 'parte', 'partir', 'paucas', 'pela', 'pelas', 'pelo', 'pelos', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante', 'perto', 'pode', 'pude', 'pôde', 'podem', 'podendo', 'poder', 'poderia', 'poderiam', 'podia', 'podiam', 'põe', 'põem', 'pois', 'ponto', 'pontos', 'por', 'porém', 'porque', 'porquê', 'posição', 'possível', 'possivelmente', 'posso', 'pouca', 'poucas', 'pouco', 'poucos', 'primeira', 'primeiras', 'primeiro', 'primeiros', 'própria', 'próprias', 'próprio', 'próprios', 'próxima', 'próximas', 'próximo', 'próximos', 'pude', 'puderam', 'quais', 'quáis', 'qual', 'quando', 'quanto', 'quantos', 'quarta', 'quarto', 'quatro', 'que', 'quê', 'quem', 'quer', 'quereis', 'querem', 'queremas', 'queres', 'quero', 'questão', 'quinta', 'quinto', 'quinze', 'relação', 'sabe', 'sabem', 'são', 'se', 'segunda', 'segundo', 'sei', 'seis', 'seja', 'sejam', 'sejamos', 'sem', 'sempre', 'sendo', 'ser', 'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'sete', 'sétima', 'sétimo', 'seu', 'seus', 'sexta', 'sexto', 'si', 'sido', 'sim', 'sistema', 'só', 'sob', 'sobre', 'sois', 'somos', 'sou', 'sua', 'suas', 'tal', 'talvez', 'também', 'tampouco', 'tanta', 'tantas', 'tanto', 'tão', 'tarde', 'te', 'tem', 'tém', 'têm', 'temos', 'tendes', 'tendo', 'tenha', 'tenham', 'tenhamos', 'tenho', 'tens', 'ter', 'terá', 'terão', 'terceira', 'terceiro', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teu', 'teus', 'teve', 'ti', 'tido', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tiveste', 'tivestes', 'toda', 'todas', 'todavia', 'todo', 'todos', 'trabalho', 'três', 'treze', 'tu', 'tua', 'tuas', 'tudo', 'última', 'últimas', 'último', 'últimos', 'um', 'uma', 'umas', 'uns', 'vai', 'vais', 'vão', 'vários', 'vem', 'vêm', 'vendo', 'vens', 'ver', 'vez', 'vezes', 'viagem', 'vindo', 'vinte', 'vir', 'você', 'vocês', 'vos', 'vós', 'vossa', 'vossas', 'vosso', 'vossos', 'zero', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '_' ]
mais_stopwords2 = str(mais_stopwords1).strip('[]')
mais_stopwords3 = mais_stopwords + mais_stopwords2
mais_stopwords3 = unidecode.unidecode(mais_stopwords3) 
lista_mais_stopwords = mais_stopwords3.split(', ')
lista_final_stopwords = advanced_sw_list1 + lista_mais_stopwords

In [93]:
#Removendo as stopwords avançadas na coluna que tem a notícia tokenizada
contador = 0
for row in df['message_norm_treatment_ssw2']:
  df['message_norm_treatment_ssw2'][contador] = [word for word in row if ((word not in lista_final_stopwords) and (len(word)>3))]
  contador+=1

## Preparação final arquivo csv

In [94]:
#Verificando tamanho do dataset
df.shape

(5201, 31)

In [95]:
#SALVANDO CSV FINAL DO TRATAMENTO
#df.to_csv("./datasetnew/treated_dataset.csv", sep=';')